# Análise exploratória dos dados

### Importando bibliotecas

In [ ]:
import pandas as pd
from datetime import datetime
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

### Ler dataset

In [2]:
df = pd.read_csv("../data/jan_train.csv")

In [3]:
df.head()

,id,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,0,2014-01-01 00:00:00.000,0,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133
1,1,2014-01-01 00:46:43.000,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468
2,2,2014-01-01 01:33:26.001,0,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768
3,3,2014-01-01 02:20:09.001,0,-9148.251857,-20651.437460,-20720.381279,0.715600,-3.373762,1.722115,-9149.620794,-20618.200201,-20765.019094,0.712437,-3.375202,1.718306
4,4,2014-01-01 03:06:52.002,0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237


### Primeiras limpezas e transformações no conjunto de dados

As seguintes ações foram realizadas:
- Drop da coluna 'id'
- Transformar em timestamp a coluna com os dados

In [4]:
df = df.drop(columns=['id'])

In [5]:
df['epoch'] = [datetime.strptime(data, '%Y-%m-%d %H:%M:%S.%f') for data in df['epoch']]

In [6]:
df.head()

,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,2014-01-01 00:00:00.000,0,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133
1,2014-01-01 00:46:43.000,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468
2,2014-01-01 01:33:26.001,0,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768
3,2014-01-01 02:20:09.001,0,-9148.251857,-20651.437460,-20720.381279,0.715600,-3.373762,1.722115,-9149.620794,-20618.200201,-20765.019094,0.712437,-3.375202,1.718306
4,2014-01-01 03:06:52.002,0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237


In [7]:
df.shape

(503227, 14)

In [8]:
df.tail()

,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
503222,2014-01-24 22:18:34.370,599,-22282.245721,2176.275085,4184.918699,1.255727,4.018148,-0.540243,-22847.208402,80.070907,4444.798405,0.870781,4.035965,-0.467196
503223,2014-01-24 22:43:25.007,599,-19551.683596,7997.489974,3224.501905,2.419800,3.729286,-0.746051,-20717.460440,6016.252299,3592.204365,2.003622,3.872352,-0.675867
503224,2014-01-24 23:08:15.645,599,-15064.616469,13079.761912,1975.820551,3.587560,3.009816,-0.920914,-16847.612994,11429.130312,2439.160526,3.186335,3.315717,-0.865266
503225,2014-01-24 23:33:06.283,599,-8953.211355,16721.787269,516.231786,4.556922,1.794477,-1.020603,-11273.779239,15659.892239,1041.308701,4.253614,2.276720,-0.996589
503226,2014-01-24 23:57:56.920,599,-1717.129724,18218.471698,-1004.927645,5.049138,0.162277,-0.996545,-4352.858942,17980.888207,-478.128914,4.945428,0.770715,-1.020298


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503227 entries, 0 to 503226
Data columns (total 14 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   epoch   503227 non-null  datetime64[ns]
 1   sat_id  503227 non-null  int64         
 2   x       503227 non-null  float64       
 3   y       503227 non-null  float64       
 4   z       503227 non-null  float64       
 5   Vx      503227 non-null  float64       
 6   Vy      503227 non-null  float64       
 7   Vz      503227 non-null  float64       
 8   x_sim   503227 non-null  float64       
 9   y_sim   503227 non-null  float64       
 10  z_sim   503227 non-null  float64       
 11  Vx_sim  503227 non-null  float64       
 12  Vy_sim  503227 non-null  float64       
 13  Vz_sim  503227 non-null  float64       
dtypes: datetime64[ns](1), float64(12), int64(1)
memory usage: 53.8 MB


In [10]:
df[df['sat_id']==0]

,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,2014-01-01 00:00:00.000,0,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133
1,2014-01-01 00:46:43.000,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468
2,2014-01-01 01:33:26.001,0,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768
3,2014-01-01 02:20:09.001,0,-9148.251857,-20651.437460,-20720.381279,0.715600,-3.373762,1.722115,-9149.620794,-20618.200201,-20765.019094,0.712437,-3.375202,1.718306
4,2014-01-01 03:06:52.002,0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,2014-01-24 20:16:45.366,0,17323.752275,-2929.599994,40015.054282,-0.066983,2.651528,-0.240019,16677.210531,5234.016160,38340.904456,-0.330504,2.634282,-0.848806
737,2014-01-24 21:03:28.366,0,16803.492692,4510.550397,38575.720092,-0.306878,2.639582,-0.792502,15395.263519,12449.405030,35146.024515,-0.586354,2.491844,-1.435062
738,2014-01-24 21:50:11.367,0,15589.464016,11758.599852,35543.718040,-0.561601,2.510341,-1.375365,13380.486475,19062.449796,30278.225301,-0.852275,2.198419,-2.039662
739,2014-01-24 22:36:54.367,0,13644.957424,18446.405418,30846.351215,-0.827062,2.233959,-1.978216,10617.516680,24593.429195,23717.669835,-1.117669,1.711798,-2.636755


In [11]:
map_df = {}
for sat_id in df['sat_id'].unique():
    map_df[sat_id] = df[df['sat_id'] == sat_id].copy()
    map_df[sat_id] = map_df[sat_id].set_index('epoch')
    map_df[sat_id].sort_index(inplace=True)
    map_df[sat_id] = map_df[sat_id].resample('45min').mean()
    map_df[sat_id] = map_df[sat_id].interpolate()
    
#     map_df[erp_name]['MM_PE_5'] = map_df[erp_name]['pressao_de_entrada'].rolling('15min').mean()
#     map_df[erp_name]['MM_PE_8'] = map_df[erp_name]['pressao_de_entrada'].rolling('25min').mean()
#     map_df[erp_name]['MM_PE_13'] = map_df[erp_name]['pressao_de_entrada'].rolling('40min').mean()

#     map_with_erp_df[erp_name]['STD_PE_5'] = map_with_erp_df[erp_name]['pressao_de_entrada'].rolling('15min').std()
#     map_with_erp_df[erp_name]['STD_PE_8'] = map_with_erp_df[erp_name]['pressao_de_entrada'].rolling('25min').std()
#     map_with_erp_df[erp_name]['STD_PE_13'] = map_with_erp_df[erp_name]['pressao_de_entrada'].rolling('40min').std()
 
#     map_with_erp_df[erp_name]['dP'] = map_with_erp_df[erp_name]['pressao_de_saida'] - map_with_erp_df[erp_name]['pressao_de_entrada']
    
#     map_with_erp_df[erp_name]['MM_dP_5'] = map_with_erp_df[erp_name]['dP'].rolling('15min').mean()
#     map_with_erp_df[erp_name]['MM_dP_8'] = map_with_erp_df[erp_name]['dP'].rolling('25min').mean()
#     map_with_erp_df[erp_name]['MM_dP_13'] = map_with_erp_df[erp_name]['dP'].rolling('40min').mean()

#     map_with_erp_df[erp_name]['STD_dP_5'] = map_with_erp_df[erp_name]['dP'].rolling('15min').std()
#     map_with_erp_df[erp_name]['STD_dP_8'] = map_with_erp_df[erp_name]['dP'].rolling('25min').std()
#     map_with_erp_df[erp_name]['STD_dP_13'] = map_with_erp_df[erp_name]['dP'].rolling('40min').std()
    
    map_df[sat_id]['x_next'] = map_df[sat_id]['x'].shift(periods=-1)
    map_df[sat_id]['y_next'] = map_df[sat_id]['y'].shift(periods=-1)
    map_df[sat_id]['z_next'] = map_df[sat_id]['z'].shift(periods=-1)
    
    map_df[sat_id].dropna(inplace =True)

In [12]:
map_df[0]

,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim,x_next,y_next,z_next
epoch,,,,,,,,,,,,,,,,
2014-01-01 00:00:00,0.0,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133,-10567.672384,1619.746066,-24451.813271
2014-01-01 00:45:00,0.0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468,-10578.684043,-10180.467460,-24238.280949
2014-01-01 01:30:00,0.0,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768,-9148.251857,-20651.437460,-20720.381279
2014-01-01 02:15:00,0.0,-9148.251857,-20651.437460,-20720.381279,0.715600,-3.373762,1.722115,-9149.620794,-20618.200201,-20765.019094,0.712437,-3.375202,1.718306,-6719.092336,-28929.061629,-14938.907967
2014-01-01 03:00:00,0.0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237,-3708.453525,-34767.115528,-7863.224747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-01-24 19:30:00,0.0,17194.282667,-10263.631730,39953.578881,0.156583,2.566601,0.277960,17261.821388,-2206.385070,39932.547083,-0.089226,2.656683,-0.292304,17323.752275,-2929.599994,40015.054282
2014-01-24 20:15:00,0.0,17323.752275,-2929.599994,40015.054282,-0.066983,2.651528,-0.240019,16677.210531,5234.016160,38340.904456,-0.330504,2.634282,-0.848806,16803.492692,4510.550397,38575.720092
2014-01-24 21:00:00,0.0,16803.492692,4510.550397,38575.720092,-0.306878,2.639582,-0.792502,15395.263519,12449.405030,35146.024515,-0.586354,2.491844,-1.435062,15589.464016,11758.599852,35543.718040


In [13]:
df.columns

Index(['epoch', 'sat_id', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz', 'x_sim', 'y_sim',
       'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim'],
      dtype='object')

In [21]:
input_model_column = ['x', 'y', 'z', 'Vx', 'Vy', 'Vz', 'x_sim', 'y_sim',
       'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']

output_model_column = ['x_next']#,'y_next','z_next']

In [22]:
number_of_split = 2
tscv = TimeSeriesSplit(n_splits=number_of_split)
train_and_test_index = {}
for key, item in map_df.items():
    train_and_test_index[key] = list(tscv.split(item))

In [23]:
def train_model(idx):
    X_train = []
    y_train = []
    for key, item in train_and_test_index.items():
        x_split = map_df[key].filter(items=input_model_column).values[item[idx][0]]
        y_split = map_df[key].filter(items=output_model_column).values[item[idx][0]]
        if len(X_train) > 0:
            X_train = np.concatenate((X_train, x_split), axis=0)
            y_train = np.concatenate((y_train, y_split), axis=0)
        else:
            X_train = x_split
            y_train = y_split        
#     y_train = column_or_1d(y_train)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    
    params = {
              "n_estimators": 100,
              "max_depth": 6,
              "max_features": 'sqrt',
              "subsample": 0.8 ,
              "min_samples_split": 20,
              "min_samples_leaf": 115,
              "learning_rate": 0.1,
              "loss": 'huber',
              "random_state": 42
    }
    gbm = GradientBoostingRegressor(**params)
    gbm.fit(X_train, y_train)
    ## SVR
    #reg_svr = SVR(gamma='scale', C=1.0, epsilon=0.2)
    #reg_svr.fit(X_train, y_train)
    ## MLP
    #reg_mlp = MLPRegressor(hidden_layer_sizes=(5, ), solver='adam', random_state=42, max_iter = 1000)
    #reg_mlp.fit(X_train, y_train)
    # return gbm, gbm, gbm, scaler
    return gbm, scaler

In [26]:
import time
start = time.time()

list_erro = []
dict_erro = {}
# for i in range(number_of_split):
for i in range(number_of_split):

    list_erro = []
    gbm, scaler = train_model(i)
    for key, item in train_and_test_index.items():
        print('\r',f'Split: {str(i)} de {number_of_split-1} ...',end='')
        
        x_test = map_df[key].filter(items=input_model_column).values[item[i][1]]
        y_test = map_df[key].filter(items=output_model_column).values[item[i][1]]
        
        x_test = scaler.transform(x_test)
        y_gbm = gbm.predict(x_test)
        #y_linear = reg_linear.predict(x_test)
        #y_svr = reg_svr.predict(x_test)
        #y_mlp = reg_mlp.predict(x_test)

        list_erro.append(np.sqrt(mean_squared_error(y_test,y_gbm)))
        dict_erro.update({'Split: '+str(i):list_erro})

results = pd.DataFrame(dict_erro, index=train_and_test_index.keys())
end = time.time()
print(f'\nEsta célula levou: {int((end-start)//60)}:{int((end-start)%60)}')

/home/barbosa/Libraries/anaconda3/envs/mlenv/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


 Split: 0 de 4 ...

/home/barbosa/Libraries/anaconda3/envs/mlenv/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


 Split: 1 de 4 ...

/home/barbosa/Libraries/anaconda3/envs/mlenv/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


 Split: 2 de 4 ...

/home/barbosa/Libraries/anaconda3/envs/mlenv/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


 Split: 3 de 4 ...

/home/barbosa/Libraries/anaconda3/envs/mlenv/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


 Split: 4 de 4 ...
Esta célula levou: 7:17


In [28]:
result_final = 0
size_total = 0
for key in range(10):
    result_final = result_final + (results.loc[key,'Split: 4']*map_df[key].shape[0])
    size_total = size_total + (map_df[key].shape[0])
    
result_final = result_final/size_total
print(f'RMSE Médio (ponderado):\n\n{result_final}')

RMSE Médio (ponderado):

3375.7667846429754


In [ ]:
best_idx = 4
gbm, scaler = train_model(best_idx)
rmse_avg = 0
for key, item in train_and_test_index.items():
    data = map_df[key].filter(items=output_model_column).index
    x_test = map_df[key].filter(items=input_model_column)
    y_test = map_df[key].filter(items=output_model_column)

    x_test = scaler.transform(x_test)
    y_gbm = gbm.predict(x_test)

    print('----------------------' + key + '----------------------')
    plt.title('Plot '+ key)
    plt.plot(data, y_test, linewidth=2, label="Real", color="blue")
    #plt.plot(data, y_pred_linear, linewidth=1, label="Predito RL" , color="green")
    #plt.plot(data, y_pred_svr, linewidth=1, label="Predito SVR", color="magenta")
    #plt.plot(data, y_pred_mlp, linewidth=1, label="Predito MLP", color="cyan")
    plt.plot(data, y_gbm, linewidth=1, label="Predito GBM", color="cyan")
    plt.legend()
    plt.show()
    plt.clf()
    rmse = np.sqrt(mean_squared_error(y_test,y_gbm))
    print('RMSE GBM: '+ str(rmse))
    print('----------------------@@@@@@@----------------------')
    rmse_avg = rmse_avg + rmse
print("RMSE Médio: "+ str(rmse_avg/len(train_and_test_index)))

/home/barbosa/Libraries/anaconda3/envs/mlenv/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
